In [1]:
!pip install transformers datasets nltk --quiet
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
import torch

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

true_df = pd.read_csv("True.csv", engine='python',on_bad_lines="skip").head(100)
fake_df = pd.read_csv("Fake.csv", engine='python',on_bad_lines="skip").head(100)

true_df['label'] = 1
fake_df['label'] = 0
df = pd.concat([true_df, fake_df], ignore_index=True)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(clean_text)

train_text, temp_text, train_label, temp_label = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42)

val_text, test_text, val_label, test_label = train_test_split(
    temp_text, temp_label, test_size=0.5, stratify=temp_label, random_state=42)

train_data = Dataset.from_dict({'text': train_text.tolist(), 'label': train_label.tolist()})
val_data = Dataset.from_dict({'text': val_text.tolist(), 'label': val_label.tolist()})
test_data = Dataset.from_dict({'text': test_text.tolist(), 'label': test_label.tolist()})

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding=True)

train_data = train_data.map(tokenize, batched=True)
val_data = val_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

trainer.train()

preds = trainer.predict(test_data)
pred_labels = preds.predictions.argmax(axis=1)

from sklearn.metrics import accuracy_score
print("Test Accuracy:", accuracy_score(test_data['label'], pred_labels))

model.save_pretrained("distilbert-fake-news")
tokenizer.save_pretrained("distilbert-fake-news")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


Step,Training Loss


Test Accuracy: 1.0


('distilbert-fake-news/tokenizer_config.json',
 'distilbert-fake-news/special_tokens_map.json',
 'distilbert-fake-news/vocab.txt',
 'distilbert-fake-news/added_tokens.json',
 'distilbert-fake-news/tokenizer.json')

In [3]:
!zip -r distilbert-fake-news.zip distilbert-fake-news/

from google.colab import files
files.download("distilbert-fake-news.zip")


updating: distilbert-fake-news/ (stored 0%)
updating: distilbert-fake-news/special_tokens_map.json (deflated 42%)
updating: distilbert-fake-news/config.json (deflated 45%)
updating: distilbert-fake-news/tokenizer_config.json (deflated 75%)
updating: distilbert-fake-news/vocab.txt (deflated 53%)
updating: distilbert-fake-news/tokenizer.json (deflated 71%)
updating: distilbert-fake-news/model.safetensors (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
model.save_pretrained("distilbert-fake-news", safe_serialization=False)


In [11]:
!cd distilbert-fake-news && zip -r ../distilbert-fake-news.zip .


  adding: special_tokens_map.json (deflated 42%)
  adding: config.json (deflated 45%)
  adding: tokenizer_config.json (deflated 75%)
  adding: vocab.txt (deflated 53%)
  adding: tokenizer.json (deflated 71%)
  adding: pytorch_model.bin (deflated 8%)
  adding: model.safetensors (deflated 8%)
